In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from scipy.stats import poisson

pd.options.mode.chained_assignment = None  # default='warn'


# Set Function
def dataCleaning(file, name):
    # Read File and Convert in DF
    df = pd.read_csv(file, delimiter=';')
    
    if(name == 'ss'):
        # Edit Columns
        df.columns = ["BRANCH_ID","PRODUCT_ID","PRODUCT_BRANCH_ID","PRODUCT","AVAILABILITY","LAST_VISITED"]
        df["LAST_VISITED"] = pd.to_datetime(df["LAST_VISITED"],format='%Y-%m-%d')
        condition = (df["BRANCH_ID"].notnull())

        return df[condition]
    
    elif(name == 'fr'):
        # Edit Columns
        df.columns = ["BRANCH_ID","PRODUCT_ID","PRODUCT_BRANCH_ID","TIMES_FOUND","TIMES_ORDERED","FOUND_RATE"]
        df["FOUND_RATE"] = df["FOUND_RATE"].str.replace(',','.')
        df["FOUND_RATE"] = df["FOUND_RATE"].astype(float)
        condition = df["BRANCH_ID"].notnull()

        return df[condition]

    elif(name == 'bk'):

        df.columns = ["PRODUCT_ID","BLOCKING_REASON","PRODUCT_NAME"]
        df["PRODUCT_ID"] = df["PRODUCT_ID"].astype(float)
        return df

def mergeFiles(ss, fr):
        
        df      =   pd.merge(ss,fr, on=['BRANCH_ID','PRODUCT_ID','PRODUCT_BRANCH_ID'],how='left')
        bkClean =   bk.drop_duplicates('PRODUCT_ID')
        df      =   pd.merge(df,bkClean, how = 'left')
        return df

def cleaningRules(ss, totalDays):
        
        date = datetime.now() - relativedelta(days=totalDays) 

        nc_R1 = ss[
                (
                        (ss['LAST_VISITED'] > date)
                    &   (ss['AVAILABILITY'].isin(['AVAILABLE','FREQUENTLY_OUT_OF_STOCK'])) 
                    &   (ss['BLOCKING_REASON'].isna())  
                    &   (ss['FOUND_RATE'] < 0.5)
                )
        ].count()[0]

        nc_R2 = ss[
                (
                        (ss['LAST_VISITED'] < date)
                    &   (ss['AVAILABILITY'].isin(['OUT_OF_STOCK'])) 
                    &   (ss['BLOCKING_REASON'].isna())  
                    &   (ss['FOUND_RATE'] < 0.5)
                )
        ].count()[0]

        nc_R3 = ss[
                (
                        (ss['LAST_VISITED'] < date)
                    &   (ss['AVAILABILITY'].isin(['OUT_OF_STOCK']))  
                    &   (ss['FOUND_RATE'] == 1)
                )
        ].count()[0]

        dataAnalysis = {
            'Type':[
                'NSS',
                'NSS',
                'NSS',
                'NSS'
            ],
            'Rules Normal Stop Send': [
                'Total Stop Send',
                'Last visited < 1 mes / Disponible para comprar / Sin bloqueo / FR < 50%: ',
                'Last visited > 1 mes / No disponible para comprar / Sin bloqueo / FR < 50%: ',
                'Last visited > 1 mes / No disponible para comprar / FR = 100%: '
            ],
            'Powered Products':[
                ss.count()[0],
                nc_R1,
                nc_R2,
                nc_R3
            ]
        }

        dataAnalysis = pd.DataFrame(dataAnalysis)  
        
        return dataAnalysis 
        
def actionList(ss, storeId, today, totalDays):

        date = today - relativedelta(days=totalDays) 

        actionListOne = ss[
                (
                        (ss['LAST_VISITED'] > date)
                    &   (ss['AVAILABILITY'].isin(['AVAILABLE','FREQUENTLY_OUT_OF_STOCK'])) 
                    &   (ss['BLOCKING_REASON'].isna())  
                    &   (ss['FOUND_RATE'] < 0.5)
                )
        ]

        actionListTwo = ss[
                (
                        (ss['LAST_VISITED'] < date)
                    &   (ss['AVAILABILITY'].isin(['OUT_OF_STOCK'])) 
                    &   (ss['BLOCKING_REASON'].isna())  
                    &   (ss['FOUND_RATE'] < 0.5)
                )
        ]

        actionListThree = ss[
                (
                        (ss['LAST_VISITED'] < date)
                    &   (ss['AVAILABILITY'].isin(['OUT_OF_STOCK']))  
                    &   (ss['FOUND_RATE'] == 1)
                )
        ]


        actionListOne   ['AVAILABILITY']    =   'UNAVAILABLE'
        actionListTwo   ['AVAILABILITY']    =   'UNAVAILABLE'
        actionListThree ['AVAILABILITY']    =   'AVAILABLE'

        bulk = pd.concat([actionListOne, actionListTwo, actionListThree], ignore_index=True)

        bulk['STORE_ID'] = storeId
        bulk['PRICE'] = ''
        bulk['SCHEDULE'] = ''

        columns = ['STORE_ID','PRODUCT_BRANCH_ID', 'PRICE', 'AVAILABILITY', 'SCHEDULE']

        bulk = bulk[columns]
        bulk.columns = bulk.columns.str.lower()

        
        # bulk.columns = ['store_id', 'product_branch_id','price','availability','schedule']

        return bulk

def overview(ss,today, days):

    date = today - relativedelta(days=days) 

    m1  =   ss[ss['AVAILABILITY'].isin(['AVAILABLE','FREQUENTLY_OUT_OF_STOCK'])].count()[0]

    m2  =   ss[ss['AVAILABILITY'].isin(['OUT_OF_STOCK'])].count()[0]

    m3  =   ss[ss['TIMES_FOUND'].notna()].count()[0]

    m4  =   ss[ss['TIMES_FOUND'].isna()].count()[0]

    m5  =   ss[ss['FOUND_RATE'] < 0.5].count()[0]

    m6  =   ss[ss['FOUND_RATE'] > 0.5].count()[0]

    m7  =   ss[ss['LAST_VISITED'] > date].count()[0]

    m8  =   ss[ss['LAST_VISITED'] < date].count()[0]

    m9  =   ss[ss['BLOCKING_REASON']   == 'AON'].count()[0]

    m10 =   ss[     (ss['BLOCKING_REASON']  == 'Tiene 2 SKU') 
                |   (ss['BLOCKING_REASON']  == 'Tiene 3 SKU')
                |   (ss['BLOCKING_REASON']  == 'Tiene 4 SKU')
            ].count()[0]

    m11 =   ss[     (ss['BLOCKING_REASON']  == 'Brands') 
                |   (ss['BLOCKING_REASON']  == 'Brands In&out')
            ].count()[0]

    m12 =   ss[     (ss['BLOCKING_REASON']  == 'Sin integración') ].count()[0]

    overviewData = {
        'Review':[
            'Disponibles para comprar (AV/FOOS)',
            'No disponibles para comprar (OOS)',
            'Con ventas',
            'Sin ventas',
            'Found Rate < 50%',
            'Found Rate > 50%',
            'Con antigüedad < 1 mes',
            'Con antigüedad > 1 mes',
            'Con bloqueo AON',
            'Con bloqueo más de 1 sku',
            'Con bloqueo brands',
            'Sin integración',
        ],
        '# PbId':[
            m1,
            m2,
            m3,
            m4,
            m5,
            m6,
            m7,
            m8,
            m9,
            m10,
            m11,
            m12
        ],
        '% PbId':[
            (m1 / ss.count()[0]) * 100,
            (m2 / ss.count()[0]) * 100,
            (m3 / ss.count()[0]) * 100,
            (m4 / ss.count()[0]) * 100,
            (m5 / ss.count()[0]) * 100,
            (m6 / ss.count()[0]) * 100,
            (m7 / ss.count()[0]) * 100,
            (m8 / ss.count()[0]) * 100,
            (m9 / ss.count()[0]) * 100,
            (m10 / ss.count()[0]) * 100,
            (m11 / ss.count()[0]) * 100,
            (m12 / ss.count()[0]) * 100
        ]
    }

    overview = pd.DataFrame(overviewData) 
    overview = overview.round({'% PbId': 2})

    overview['% PbId'] = [ "{}%".format(v) for v in overview['% PbId'] ]

    return overview


storeId     =   4
week        =   '17'
cleaning    =   '02'
days        =   32

today = datetime.now()

ssFile = 'FILES/W'+week+'/'+cleaning+'_SS_W'+week+'.csv'
frFile = 'FILES/W'+week+'/'+cleaning+'_FR_W'+week+'.csv'
bkFile = 'FILES/W'+week+'/'+cleaning+'_BQ_W'+week+'.csv'

ss  =   dataCleaning(ssFile, 'ss' )
fr  =   dataCleaning(frFile, 'fr' )
bk  =   dataCleaning(bkFile, 'bk' )


ssM     =   mergeFiles(ss,fr)

ssResults   =   cleaningRules(ssM, days)
bulk        =   actionList(ssM,storeId, today, days)
overview = overview(ssM, today, days)



today = str(datetime.today().strftime("%Y-%m-%d"))

fr

# START EXPORT FILES

bulk.to_excel('FILES/W'+week+'/'+today+'_BULK_JUMBO_CLEANING.xlsx', index=False) # FILE TO UPLOAD TO THE BULK

analysis = pd.ExcelWriter('FILES/W'+week+'/'+today+'_ANALYSIS_STOP_SEND_JUMBO.xlsx', engine='xlsxwriter') # CREATE MULTI SHEETS FILES

overview.to_excel(analysis, sheet_name='OVERVIEW',index=False)              # CREATE OVERVIEW SHEET
ssResults.to_excel(analysis, sheet_name='TOTAL ACTIONABLES',index=False)    # CREATE TOTAL ACTIONABLES SHEET
bulk.to_excel(analysis, sheet_name='POWERED PRODUCTBRANCHS',index=False)    # CREATE POWERED PRODUCTBRANCHS SHEET
ssM.to_excel(analysis, sheet_name='FULL FILE',index=False)     # CREATE POWERED PRODUCTBRANCHS SHEET
analysis.save() # SAVE MULTI SHEET FILE CHANGES

print('Successful Export')



Successful Export
